# Case Study: Recidivism

*This material owes a special thanks to Arnav Snood who helped replicate pieces of the original Pro
Publica analysis, developed regression models, and explored various pieces of the data*

**Prerequisites**

- [matplotlib Introduction](pandas/matplotlib.ipynb)  
- [Visualization Rules](visualization_rules.ipynb)  
- Regression  


**Outcomes**

- See an end-to-end data science exercise  
- Application of regression  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import (
    linear_model, metrics, neural_network, pipeline, preprocessing, model_selection
)

%matplotlib inline

## Introduction to Recidivism

Recidivism is the tendency for an individual who has previously committed a crime to commit another crime
in the future. One key input to a judge’s sentencing decision is how likely a given convict is to re-offend, or recidivate

In an effort to assist the legal system with sentencing guidelines, data scientists have attempted
to predict an individual’s risk of recidivism from known observables. Some are concerned that this process
may exhibit prejudice, either through biased inputs or through statistical discrimination. For example,

1. 
  <dl style='margin: 20px 0;'>
  <dt>Biased inputs: Imagine that a judge often writes harsher sentences to people of a particular</dt>
  <dd>
  race or gender. If an algorithm is trained to reproduce the sentences of this judge,
  the bias will be propagated by the algorithm  
  </dd>
  
  </dl>
  
1. 
  <dl style='margin: 20px 0;'>
  <dt>Statistical discrimination: Imagine that two variables (say race and income) are correlated, and one of them (say income)</dt>
  <dd>
  is correlated with the risk of recidivism. If income is unobserved, then an otherwise unbiased
  method would discriminate based on race, even if race has nothing to say about recidivism after
  controlling for income  
  </dd>
  
  </dl>
  


This has given rise to serious discussions about the moral obligations data scientists have to
those who are affected by their tools. We will not take a stance today on our moral obligations, but
we believe this is an important precursor to any statistical work with public policy applications

One predictive tool used by various courts in the United States is
called COMPAS (Correctional Offender Management Profiling for Alternative Sanctions). We will be
following a [Pro Publica](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)
article that analyzes the output of COMPAS. The findings of the article include:

- Black defendants were often predicted to be at a higher risk of recidivism than they actually were  
- White defendants were often predicted to be less risky than they were  
- When controlling for prior crimes, future recidivism, age, and gender, black defendants were 45
  percent more likely to be assigned higher risk scores than white defendants  
- Black defendants were twice as likely as white defendants to be misclassified as being a higher
  risk of violent recidivism  
- Even when controlling for prior crimes, future recidivism, age, and gender, black defendants were
  77 percent more likely to be assigned higher risk scores than white defendants  

## Data Description

The authors of this article filed a public records request with the Broward County Sheriff’s office
in Florida. Luckily for us, they did a significant amount of the legwork which is described in this
[methodology article](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm)

We download the data below:

In [ ]:
data_url = "https://raw.githubusercontent.com/propublica/compas-analysis"
data_url += "/master/compas-scores-two-years.csv"

df = pd.read_csv(data_url)
df.head()

We summarize some of the variables that we will use

- `first`: An individual’s first name  
- `last`: An individual’s last name  
- `sex`: An individual’s sex  
- `age`: An individual’s age  
- 
  <dl style='margin: 20px 0;'>
  <dt>`race`: An individual’s race. It takes values of Caucasian, Hispanic, African-American, Native</dt>
  <dd>
  American, Asian, or Other  
  </dd>
  
  </dl>
  
- `priors_count`: Number of previous arrests  
- `decile_score`: The COMPAS risk score  
- `two_year_recid`: Whether the individual had been jailed for a new crime in next two years  

## Descriptive Statistics

The first thing we do with our data is to drop any classes without “enough” observations. One of our
focuses will be on inter-race differences in scores and recidivism, so we only keep data on races
with at least 500 observations in our data set:

In [ ]:
race_count = df.groupby(["race"])["name"].count()
at_least_500 = list(race_count[race_count > 500].index)
print("The following race have at least 500 observations:", at_least_500)
df = df.loc[df["race"].isin(at_least_500), :]

Next, we explore the remaining data using plots and tables

### Age, Sex, and Race

Let’s look at how the dataset is broken down into age, sex, and race

In [ ]:
def create_groupcount_barplot(df, group_col, figsize, **kwargs):
    "call df.groupby(group_col), then count number of records and plot"
    counts = df.groupby(group_col)["name"].count().sort_index()

    fig, ax = plt.subplots(figsize=figsize)
    counts.plot(kind="bar", **kwargs)

    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.set_xlabel("")
    ax.set_ylabel("")

    return fig, ax

In [ ]:
age_cs = ["Less than 25", "25 - 45", "Greater than 45"]
df["age_cat"] = pd.Categorical(df["age_cat"], categories=age_cs, ordered=True)
fig, ax = create_groupcount_barplot(df, "age_cat", (14, 8), color="DarkBlue", rot=0)

In [ ]:
sex_cs = ["Female", "Male"]
df["sex"] = pd.Categorical(df["sex"], categories=sex_cs, ordered=True)
create_groupcount_barplot(df, "sex", (6, 8), color="DarkBlue", rot=0)

In [ ]:
race_cs = ["African-American", "Caucasian", "Hispanic"]
df["race"] = pd.Categorical(df["race"], categories=race_cs, ordered=True)
create_groupcount_barplot(df, "race", (12, 8), color="DarkBlue", rot=0)

We learn from this is that the population that we are looking at is mostly between 25-45, male, and
is mostly African-American or Caucasian

### Recidivism

We now look into how recidivism is split across groups.

In [ ]:
recid = df.groupby(["age_cat", "sex", "race"])["two_year_recid"].mean().unstack(level="race")
recid

In the table we see that the young have higher recidivism rates than the old, except for among
Caucasian females. Also, African-American males are at a particularly high risk of recidivism even
as they get older

### Risk Scores

Each individual in the dataset was assigned a `decile_score` ranging from 1 to 10

This score represents the perceived risk of recidivism with 1 being the lowest risk and 10 being the highest

We show a bar plot of all decile scores below

In [ ]:
create_groupcount_barplot(df, "decile_score", (12, 8), color="DarkBlue", rot=0)

How do these scores differ by race?

In [ ]:
dfgb = df.groupby("race")
race_count = df.groupby("race")["name"].count()

fig, ax = plt.subplots(3, figsize=(14, 8))

for (i, race) in enumerate(["African-American", "Caucasian", "Hispanic"]):
    (
        (dfgb
            .get_group(race)
            .groupby("decile_score")["name"].count() / race_count[race]
        )
        .plot(kind="bar", ax=ax[i], color="#353535")
    )
    ax[i].set_ylabel(race)
    ax[i].set_xlabel("")
    # set equal y limit for visual comparison
    ax[i].set_ylim(0, 0.32)

fig.suptitle("Score Frequency by Race")

While Caucasians and Hispanics both see majority of their score distribution on low values,
African-Americans are almost equally likely to receive any score

### Risk Scores and Recidivism

Now we can explore the relationship between the risk score and actual two year recidivism

The first measure we look at is the frequency of recidivism by decile score – These numbers
tell us what percentage of people assigned a particular risk score committed a new crime within two
years of being released

In [ ]:
df.groupby("decile_score")["two_year_recid"].mean()

Let’s also look at the correlation correlation

In [ ]:
df[["decile_score", "two_year_recid"]].corr()

The percentage of people committing a new crime is increasing in the risk score and there is a
positive correlation (~0.35)

This is good news – it means that the score is producing at least some signal about an individual’s recidivism risk

One of the key critiques from Pro Publica, though, was that the inaccuracies were nonuniform — that is, the tool was
systematically wrong about certain populations

Let’s now separate the correlations by race and see what happens

In [ ]:
recid_rates = df.pivot_table(index="decile_score", columns="race", values="two_year_recid")

recid_rates

Or, in plotted form,

In [ ]:
fig, ax = plt.subplots(3, sharex="all")

for (i, _race) in enumerate(["African-American", "Caucasian", "Hispanic"]):
    _rr_vals = recid_rates[_race].values

    ax[i].bar(np.arange(1, 11), _rr_vals, color="#c60000")
    ax[i].bar(np.arange(1, 11), 1 - _rr_vals, bottom=_rr_vals, color="#353535")
    ax[i].set_ylabel(_race)
    ax[i].spines["left"].set_visible(False)
    ax[i].spines["right"].set_visible(False)
    ax[i].spines["top"].set_visible(False)
    ax[i].spines["bottom"].set_visible(False)
    ax[i].yaxis.tick_right()
    ax[i].xaxis.set_ticks_position("none")

fig.suptitle("Recidivism Rates by Race")

## Regression

In what follows, we will be doing something slightly different than what was done in the Pro Publica
article

First, we will explore what happens when we try to predict the COMPAS risk scores using the
observable data that we have

Second, we will use binary probability models to try and predict whether an individual is at risk of
recidivism. We will do this first using the COMPAS risk scores, and then afterwards we will try to write our own
model based on raw observables, like age, race and sex

### Preprocessing

We would like to use some features that are inherently non-numerical such as sex, age group, and
race in our model. Before we can do that we need to encode these string values as numerical values
so our machine learning algorithms can understand them – An econometrician would call this,
creating dummy variables. `sklearn` can automatically do this for us using `OneHotEncoder`

The main idea of how this works is to make one column for each possible value of a categorical
variable then we will set just one of these columns equal to a 1 if the observation has that
column’s category, and set all other columns to 0.

Let’s do an example

Imagine we have the array below

In [ ]:
sex = np.array([["Male"], ["Female"], ["Male"], ["Male"], ["Female"]])

The way to encode this would be to create the array below

In [ ]:
sex_encoded = np.array([
    [0.0, 1.0],
    [1.0, 0.0],
    [0.0, 1.0],
    [0.0, 1.0],
    [1.0, 0.0]
])

Using `sklearn` it would be

In [ ]:
ohe = preprocessing.OneHotEncoder(sparse=False)
sex_ohe = ohe.fit_transform(sex)

# This should shows 0s!
sex_ohe - sex_encoded

We will use this encoding trick below as we create our data

### Predicting COMPAS Scores

First we proceed by creating the `X` and `y` inputs into a manageable format. We encode the categorical
variables using the `OneHotEncoder` described above, and then merge that with the non-categorical data. Finally,
we split the data into training and validation (test) subsets

In [ ]:
def prep_data(df, continuous_variables, categories, y_var, test_size=0.15):

    ohe = preprocessing.OneHotEncoder(sparse=False)

    y = df[y_var].values
    X = np.zeros((y.size, 0))

    # Add continuous variables if exist
    if len(continuous_variables) > 0:
        X = np.hstack([X, df[continuous_variables].values])

    if len(categories) > 0:
        X = np.hstack([X, ohe.fit_transform(df[categories])])

    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=test_size, random_state=42
    )

    return X_train, X_test, y_train, y_test

As we proceed, our goal will be to see which variables are most important for predicting the COMPAS
scores. As we estimate these models, one of our metrics for success will be mean absolute error
(MAE)

In [ ]:
def fit_and_report_maes(mod, X_train, X_test, y_train, y_test, y_transform=None, y_inv_transform=None):
    if y_transform is not None:
        mod.fit(X_train, y_transform(y_train))
    else:
        mod.fit(X_train, y_train)

    yhat_train = mod.predict(X_train)
    yhat_test = mod.predict(X_test)

    if y_transform is not None:
        yhat_train = y_inv_transform(yhat_train)
        yhat_test = y_inv_transform(yhat_test)

    return dict(
        mae_train=metrics.mean_absolute_error(y_train, yhat_train),
        mae_test=metrics.mean_absolute_error(y_test, yhat_test)
    )

Let’s begin with a simple linear model which uses just prior arrests

In [ ]:
X_train, X_test, y_train, y_test = prep_data(
    df, ["priors_count"], [], "decile_score"
)

fit_and_report_maes(linear_model.LinearRegression(), X_train, X_test, y_train, y_test)

This simple model obtains a MAE of about 2 for both the test data and training data. This means, on
average, that our model can predict the COMPAS score (which ranges from 1-10) within about 2 points

While the MAE is about 2, it’s often useful to know what the errors on our prediction model look
like. Below we create a histogram which shows the distribution of these errors – In our case we
take the difference between predicted value and actual value, so a positive value means that we
overpredicted the COMPAS score and a negative value means we underpredicted it

In [ ]:
lr_model = linear_model.LinearRegression()
lr_model.fit(X_train, y_train)

yhat_train = lr_model.predict(X_train)
yhat_test = lr_model.predict(X_test)

fig, ax = plt.subplots(1, 2, figsize=(12, 4), sharey="all")

ax[0].hist(yhat_train - y_train, density=True)
ax[0].set_title("Training Data")
ax[1].hist(yhat_test - y_test, density=True)
ax[1].set_title("Test Data")

In both cases, there are long left tails of errors meaning there are features of the data that might
be missing that would allow us to accurately predict the scores

The first thing we might consider investigating is whether there are non-linearities in how the
number of priors enters the COMPAS score

First we try using polynomial features in our exogenous variables

In [ ]:
X_train, X_test, y_train, y_test = prep_data(
    df, ["priors_count"], [], "decile_score"
)

# Transform data to quadratic
pf = preprocessing.PolynomialFeatures(2, include_bias=False)
X_train = pf.fit_transform(X_train)
X_test = pf.fit_transform(X_test)

fit_and_report_maes(linear_model.LinearRegression(), X_train, X_test, y_train, y_test)

We don’t see a very significant increase in performance, so we also try using log on the endogenous
variables

In [ ]:
X_train, X_test, y_train, y_test = prep_data(
    df, ["priors_count"], [], "decile_score"
)

fit_and_report_maes(
    linear_model.LinearRegression(), X_train, X_test, y_train, y_test,
    y_transform=np.log, y_inv_transform=np.exp
)

Still no improvement… The next natural thing is to add more features to our regression

In [ ]:
X_train, X_test, y_train, y_test = prep_data(
    df, ["priors_count"], ["age_cat", "race", "sex"], "decile_score"
)

fit_and_report_maes(linear_model.LinearRegression(), X_train, X_test, y_train, y_test)

By allowing for indicator variables on age, race, and sex, we are able to slightly improve the MAE.
The errors also seem to have a less extreme tail

In [ ]:
X_train, X_test, y_train, y_test = prep_data(
    df, ["priors_count"], ["age_cat", "race", "sex"], "decile_score"
)

lr_model = linear_model.LinearRegression()
lr_model.fit(X_train, y_train)

yhat_train = lr_model.predict(X_train)
yhat_test = lr_model.predict(X_test)

fig, ax = plt.subplots(1, 2, figsize=(12, 4), sharey="all")

ax[0].hist(yhat_train - y_train, density=True)
ax[0].set_title("Training Data")
ax[1].hist(yhat_test - y_test, density=True)
ax[1].set_title("Test Data")

The coefficients are listed below:

In [ ]:
names = [
    "priors_count", "Less than 25", "25-45", "Greater than 45", "African-American",
    "Caucasian", "Hispanic", "Female", "Male"
]
for (_name, _coef) in zip(names, lr_model.coef_[0, :]):
    print(_name, ": ", _coef)

What stands out to you about these coefficients?

<blockquote>

**Check for understanding**

Can you develop a model that performs better at mimicking their risk scores?


</blockquote>

### Binary Probability Models

Binary probability models are used to model the probability of an event occurring. The output of
this family of model is the probability that an event of interest occurs. With this probability in
hand, the researcher chooses an acceptable cutoff (perhaps 0.5) above which the event is predicted
to occur

>**Note**
>
>Binary probability models can be thought of as a special case of
classification. In classification we are given a set of features and asked
to predict one of a finite number of discrete labels. We will learn more
about classification in an upcoming lecture!

In our example, we will be interested in how the COMPAS scores do at predicting recidivism and how
their ability to predict depends on race or sex

To assist us in evaluating the performance of various models we will use a new
metric called the *confusion matrix*. Scikit-learn knows how to compute this
metric, and also provides a good description of what is computed. Let’s see
what they have to say

In [ ]:
help(metrics.confusion_matrix)

In [ ]:
def fit_and_report_cm(mod, X_train, X_test, y_train, y_test):
    mod.fit(X_train, y_train)
    return dict(
        cm_train=metrics.confusion_matrix(y_train, mod.predict(X_train)),
        cm_test=metrics.confusion_matrix(y_test, mod.predict(X_test))
    )

We will start by using logistic regression using only `decile_score` as a feature and then examine
how the confusion matrices differ by race and sex

In [ ]:
groups = [
    "overall", "African-American", "Caucasian", "Hispanic", "Female", "Male"
]

ind = [
    "Percent_of_NoRecid_given_LowRisk", "Percent_of_Recid_given_LowRisk",
    "Percent_of_NoRecid_given_HighRisk", "Percent_of_Recid_given_HighRisk"
]
output = pd.DataFrame(index=ind, columns=groups)

for group in groups:
    if group in ["African-American", "Caucasian", "Hispanic"]:
        df_subset = df.query("race == @group")
    elif group in ["Female", "Male"]:
        df_subset = df.query("sex == @group")
    else:
        df_subset = df

    X_train, X_test, y_train, y_test = prep_data(
        df_subset, ["decile_score"], [], "two_year_recid", test_size=0.25
    )

    cm = fit_and_report_cm(
        linear_model.LogisticRegression(), X_train, X_test, y_train, y_test
    )["cm_test"]

    # Compute fraction for which the guess is correct
    total_lowrisk = cm[:, 0].sum()
    total_highrisk = cm[:, 1].sum()
    vals = np.array([
        cm[0, 0] / total_lowrisk, cm[1, 0] / total_lowrisk,
        cm[0, 1] / total_highrisk, cm[1, 1] / total_highrisk
    ])
    output.loc[:, group] = vals

output

`output` contains information on the fraction of true negatives, false negatives, false positives,
and true positives. What do you see?